Convert TIFFs to Zarr
https://pythonhosted.org/python-bioformats/#reading-images

In [ ]:
import math
import numpy as np
import os
import sys
import xmltodict
from zarr.hierarchy import open_group

import javabridge
import bioformats
from omero_zarr import raw_pixels

javabridge.start_vm(class_path=bioformats.JARS)

In [ ]:
filepath = ('/uod/idr/filesets/idr0099-jain-beetlelightsheet/20201001-ftp/'
            'Akanksha_Jain_22-06-16_Tc-Squash-eGFP/TP{index}_Ch0_Ill0_Ang1,2,3,4,5.tif')
maxindex = 212
dtype = np.uint16

Get dimensions from the first image

In [ ]:
f = filepath.format(index=0)
m = xmltodict.parse(bioformats.get_omexml_metadata(f))
p = bioformats.load_image(f, c=0, z=0, t=0, rescale=False)
[size_x, size_y, size_z, size_c, size_t] = [
    int(m['OME']['Image']['Pixels'][f'@Size{f}']) for f in 'XYZCT']
dtype = m['OME']['Image']['Pixels']['@Type']
print([size_x, size_y, size_z, size_c, size_t], dtype)

In [ ]:
zct_list = []
size_t = maxindex + 1
# Keep going until smallest dimension is in [96,192)
level_count = int(math.log2((max(size_x, size_y, 96) - 1) / 96)) + 1

def plane_generator():
    for t in range(size_t):
        f = filepath.format(index=t)
        sys.stdout.write(f'{f}\n')
        for c in range(size_c):
            for z in range(size_z):
                if os.path.exists(f):
                    p = bioformats.load_image(f, c=c, z=z, t=0, rescale=False)
                    yield p.astype(dtype)
                else:
                    yield None

In [ ]:
root = open_group('22-06-16_Tc-Squash-eGFP', mode='w')
levels = raw_pixels.add_raw_image(
    planes=plane_generator(),
    size_x=size_x,
    size_y=size_y,
    size_z=size_z,
    size_c=size_c,
    size_t=size_t,
    level_count=level_count,
    d_type=dtype,
    parent=root)
print(f'Levels: {levels}')
raw_pixels.add_group_metadata(root, None, levels)


In [ ]:
javabridge.kill_vm()